In [3]:

import numpy as np
import math
from os.path import join,exists
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import Merge
from keras.layers.convolutional import Conv2D,MaxPooling2D,MaxPooling1D, Conv1D
from keras.optimizers import Adadelta,RMSprop
#from hyperas.distributions import choice, uniform, conditional
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from random import randint
from sklearn.cross_validation import train_test_split
from keras import backend as K
K.set_image_dim_ordering('th')

def parse_seq(a,maxlen):
    encode = {'A':[1,0,0,0],'C':[0,1,0,0],'G':[0,0,1,0],'T':[0,0,0,1],'U':[0,0,0,1],'N':[0,0,0,0]}
    out = np.zeros([maxlen,4])
    for idx, nt in enumerate(a):
        out[idx]=encode[nt]
    return out



In [6]:
fi = 'input100k'
maxlen_mir = 0
for l in open(fi, 'r'):
    (a, b) = l.strip().split("\t")
    if len(a)> maxlen_mir:
            maxlen_mir = len(a)
print maxlen_mir     

28


In [ ]:
def parse_seq(a,maxlen):
    encode = {'A':[1,0,0,0],'C':[0,1,0,0],'G':[0,0,1,0],'T':[0,0,0,1],'U':[0,0,0,1],'N':[0,0,0,0]}
    out = np.zeros([maxlen,4])
    for idx, nt in enumerate(a):
        out[idx]=encode[nt]
    return out

In [8]:
def parse_seq1D(a,maxlen):
    encode = {'A':1,'C':2,'G':3,'T':4,'U':5,'N':0}
    out = np.zeros([maxlen,4])
    for idx, nt in enumerate(a):
        out[idx]=encode[nt]
    return out

In [5]:
fi = 'input100k'
len1k_gene = 0
for l in open(fi, 'r'):
    (a, b) = l.strip().split("\t")
    if len(b)<7000 and len(b)>100:
            len1k_gene+=1
print len1k_gene

84909


In [6]:
out_mir = []
out_gene = []
maxlen_mir =28
fi = 'control10k'
len10k_gene = 0
print fi
for l in open(fi, 'r'):
    (a, b) = l.strip().split("\t")

    if len(b)<7000:
        out_mir.append(parse_seq(a,maxlen_mir))
        out_gene.append(parse_seq(b,7000))
        len10k_gene+=1

print len10k_gene

out_mir=np.array(out_mir)
np.save('out_mir_control.npy', out_mir)

out_gene = np.array(out_gene)
np.save('out_gene_control.npy', out_gene)

control10k
9835


In [4]:
out_mir= np.load('out_mir_input.npy')
out_gene = np.load('out_gene_input.npy')
#print out_mir.shape
out_mir2 = out_mir.reshape(8500,28,4)
out_gene2 = out_gene.reshape(8500,7000,4)

In [16]:
def run_model(X_train_g,X_train_m, Y_train, X_test_g,X_test_m, Y_test, nb_epoch):
   
    DROPOUT = 0.5 
    
    gmodel = Sequential()
    gmodel.add(Conv1D(20,4, input_shape=(7000,4),activation='relu'))
    gmodel.add(MaxPooling1D(2))
    gmodel.add(Flatten())
    gmodel.add(Dense(32,activation='relu'))
    gmodel.add(Dropout(DROPOUT))
    
    mmodel = Sequential()
    mmodel.add(Conv1D(20,4, input_shape=(28,4),activation='relu'))
    mmodel.add(MaxPooling1D(2))
    mmodel.add(Flatten())
    mmodel.add(Dense(32,activation='relu'))
    mmodel.add(Dropout(DROPOUT))
    
    merged = Merge([gmodel, mmodel], mode = 'concat')
    modelmg = Sequential()
    modelmg.add(merged)
    #modelmg.add(Dense(32,activation='relu'))
    modelmg.add(Dropout(DROPOUT))
    modelmg.add(Dense(1))
    modelmg.add(Activation('softmax'))
    print ("layers are added")

    myoptimizer = RMSprop(lr=0.01, rho=0.9, epsilon=1e-06) #{{choice([0.01,0.001,0.0001])}}
    mylossfunc = 'binary_crossentropy'
    modelmg.compile(loss=mylossfunc, optimizer=myoptimizer,metrics=['accuracy'])
    print ("compiled")
    modelmg.fit([X_train_g,X_train_m], Y_train, batch_size=100, nb_epoch=nb_epoch,validation_split=0.1)
    print ("fitted")
    score, acc = modelmg.evaluate([X_test_g,X_test_m], Y_test)
    print ("evaluated")
    print('Test accuracy:', acc,'test loss',score)
    #return {'loss': score, 'status': STATUS_OK,'model':(model.to_json(),myoptimizer,mylossfunc)}


In [8]:
print (out_mir2.shape)

(8500, 28, 4)


In [18]:
out_mir.reshape(4,1,28,8500)

array([[[[ 1.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  1.,  0.,  0.],
         ..., 
         [ 0.,  1.,  0., ...,  0.,  0.,  1.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  1., ...,  0.,  0.,  0.]]],


       [[[ 1.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  1.],
         [ 0.,  0.,  0., ...,  0.,  1.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  1.],
         [ 0.,  0.,  1., ...,  0.,  0.,  0.]]],


       [[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  1.],
         ..., 
         [ 0.,  0.,  1., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  1., ...,  0.,  0.,  0.]]],


       [[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.

In [140]:
out_gene2.shape

(8500, 7000, 4)

In [12]:
# Generation of 'negative' genes by simply shuffling the original genes' set
from numpy.random import permutation
genes_1 = out_gene2
perm1 = permutation(genes_1.shape[0])
genes_0 = genes_1[perm1]

genes = np.concatenate((genes_1, genes_0))
                        #, genes_02, genes_03
                        #, genes_04))
targets_1 = np.empty(genes_1.shape[0])
targets_1.fill(1) 
targets_0 = np.zeros(genes_1.shape[0])
targets = np.concatenate((targets_1, targets_0))
mirs = np.concatenate((out_mir2, out_mir2))
                       #, mirs_1, mirs_1
                       #, mirs_1))

perm = permutation(genes.shape[0])
genes = genes[perm]
mirs = mirs[perm]
targets = targets[perm]
# For test
train_size = int(0.8*genes_1.shape[0])
genes_t = genes[train_size:-1]
mirs_t = mirs[train_size:-1]
targets_t = targets[train_size:-1]

genes = genes[0:train_size]
mirs = mirs[0:train_size]
targets = targets[0:train_size]

In [18]:
run_model(genes,mirs,targets,genes_t,mirs_t,targets_t,nb_epoch=30)

/home/kira/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:19: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/kira/.local/lib/python3.5/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


layers are added
compiled
Train on 6120 samples, validate on 680 samples
Epoch 1/30
6120/6120 [==============================] - 20s 3ms/step - loss: 7.9816 - acc: 0.4993 - val_loss: 7.4320 - val_acc: 0.5338
Epoch 2/30
6120/6120 [==============================] - 20s 3ms/step - loss: 7.9816 - acc: 0.4993 - val_loss: 7.4320 - val_acc: 0.5338
Epoch 3/30
6120/6120 [==============================] - 19s 3ms/step - loss: 7.9816 - acc: 0.4993 - val_loss: 7.4320 - val_acc: 0.5338
Epoch 4/30
6120/6120 [==============================] - 20s 3ms/step - loss: 7.9816 - acc: 0.4993 - val_loss: 7.4320 - val_acc: 0.5338
Epoch 5/30
6120/6120 [==============================] - 20s 3ms/step - loss: 7.9816 - acc: 0.4993 - val_loss: 7.4320 - val_acc: 0.5338
Epoch 6/30
6120/6120 [==============================] - 20s 3ms/step - loss: 7.9816 - acc: 0.4993 - val_loss: 7.4320 - val_acc: 0.5338
Epoch 7/30
6120/6120 [==============================] - 20s 3ms/step - loss: 7.9816 - acc: 0.4993 - val_loss: 7.4320 